Copied example code from DIVAnd.jl package

In [1]:
using DIVAnd
using Dates

In [2]:
varname = "Salinity"
filename = "WOD-Salinity.nc"

if !isfile(filename)
    download("https://dox.ulg.ac.be/index.php/s/PztJfSEnc8Cr3XN/download",filename)
end


In [3]:
value,lon,lat,depth,time,ids = DIVAnd.loadobs(Float64,filename,"Salinity")

DIVAnd.checkobs((lon,lat,depth,time),value,ids)


              minimum and maximum of obs. dimension 1: (3.0418334007263184, 11.8100004196167)

┌ Info: Checking ranges for dimensions and observations
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\obsstat.jl:75



              minimum and maximum of obs. dimension 2: (42.0, 44.0)
              minimum and maximum of obs. dimension 3: (-0.0, 2762.0)
              minimum and maximum of obs. dimension 4: (DateTime("1892-09-25T00:00:00"), DateTime("2017-10-02T06:14:02.761"))
                          minimum and maximum of data: (0.25, 40.0)


In [14]:
# Does sel stand for selector?
sel = (Dates.month.(time) .== 1)
x = (lon[sel],lat[sel],depth[sel]);
v = value[sel]
z = [0.,10,100,200,300,400,500,700,1000,1500]

# println(x) #See how the tuple is structured
println(x[:,1:5])

LoadError: MethodError: no method matching getindex(::Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}}, ::Colon, ::UnitRange{Int64})
[0mClosest candidates are:
[0m  getindex(::Tuple, ::Colon) at tuple.jl:33
[0m  getindex(::Tuple, [91m::Int64[39m) at tuple.jl:29
[0m  getindex(::Tuple, [91m::Real[39m) at tuple.jl:30
[0m  ...

In [7]:
println(size(lon[sel]))
println(size(lat[sel]))
println(size(depth[sel]))

println(depth[sel][187467:187468+50])

(225570,)
(225570,)
(225570,)
[2601.0, 2695.0, 2.0, 3.0, 10.0, 83.0, 114.0, 216.0, 319.0, 421.0, 524.0, 626.0, 729.0, 832.0, 933.0, 1036.0, 1138.0, 1239.0, 1339.0, 1439.0, 1539.0, 1604.0, 1641.0, 1741.0, 1881.0, 1942.0, 2042.0, 2142.0, 2242.0, 2343.0, 2443.0, 2543.0, 2612.0, 5.0, 11.0, 22.0, 29.0, 44.0, 57.0, 80.0, 117.0, 220.0, 322.0, 344.0, 425.0, 528.0, 631.0, 733.0, 836.0, 939.0, 1040.0, 1143.0]


The cell above shows that the file contains multiple profiles since the depth contains multiple increasing ranges. The profiles do not all contain observations at the same depths.

In [6]:
# findmax() finds the maximum element and its position in an array
println(findmax(depth[sel]))
println(findmax(lon[sel]))
println(findmax(lat[sel]))

print(typeof(lon))

(2695.0, 187468)
(11.649999618530273, 183862)
(44.0, 189674)
Vector{Float64}

In [8]:
lenxy,infoxy = DIVAnd.fithorzlen(x,v,z);

┌ Info: Data points at z=0.0: 18551, horz. correlation length: 0.29424930868581095 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=10.0: 21948, horz. correlation length: 0.2703533603209832 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=100.0: 30934, horz. correlation length: 0.1440978411874747 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=200.0: 26778, horz. correlation length: 0.10758444248871805 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Warning: Too few data. Will use guesses (np = 9, RLz = 0.23018796056824598, )
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:891
┌ Info: Data points at z=300.0: 17596, horz. correlation length: 0.23018796056824598 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDB

In [11]:
println(lenxy)
println(size(lenxy))

[0.20473074440480052, 0.20403972595666858, 0.16663107706437486, 0.12999927378172726, 0.119633749370118, 0.11544481872615496, 0.11314058096901758, 0.11181089491400162, 0.11102889018379454, 0.12086689637528193]
(10,)


The function `fithorzlen()` spits out an array of correlation lengths for each depth that the user wants to interpolate to?

# Try out on Oxy data...

In [30]:
using CSV
using DataFrames
using DelimitedFiles
using Statistics

In [ ]:
# Load the standard levels into an array or something
slpath = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\",
    "lu_docs\\WOA_Standard_Depths.txt")

sl = readdlm(slpath)

# deleteat!(sl, findall(x->x=="",sl))

# Reshape matrix into 1D array
sl_arr = reshape(sl, size(sl)[1] * size(sl)[2])

# Remove all empty strings from array
sl_arr = deleteat!(sl_arr, findall(x->x=="", sl_arr))

# Remove comma from standard level array elements and convert them to ints
# List comprehension
# parse() converts strings to Int32 data type
sl_int = [parse(Int32, x[1:(length(x) - 1)]) for x=sl_arr]

# Sort the array from smallest to largest
sl_int = sort(sl_int)

In [46]:
# Set coverage
years = [1991:1:2020;]  # Equivalent to numpy.arange()
szns = ["JFM", "AMJ", "JAS", "OND"]

# Initialize a 2D array to hold the mean correlation length from fithorzlen
lenxy_mtx = zeros(Float32, length(years), length(szns))

30×4 Matrix{Float32}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 ⋮              
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [47]:
for i=1:length(years)
    for j=1:length(szns)
        # Observational data
        fpath = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\",
            "data\\value_vs_depth\\15_sep_by_year_szn_obslevel\\Oxy_", 
            years[i], "_", szns[j], "_obslevel.csv")

        # Pipe to dataframe
        df = CSV.File(fpath) |> DataFrame;

        # println(first(df, 5))

        # Limit the array of standard levels to the standard levels within the
        # range of the input oxygen data

        # returns a tuple (min, index)
        sl_int_min = deleteat!(
            sl_int, findall(x->x < findmin(df[!, :Depth_m])[1], sl_int))

        # Delete depths greater than 50m
        sl_final = deleteat!(
            sl_int_min, findall(x -> x > 100, sl_int))

        # Now try the fithorzlen()
        x_oxy = (df[!, :Longitude], df[!, :Latitude], df[!, :Depth_m])
        v_oxy = df[!, :Value]
        z_oxy = sl_final

        lenxy_oxy,infoxy_oxy = DIVAnd.fithorzlen(x_oxy, v_oxy, z_oxy);

        println(i, j, lenxy_oxy)
        println(mean(lenxy_oxy))
        
        lenxy_mtx[i, j] = mean(lenxy_oxy)
    end
end

┌ Info: Data points at z=5: 1014, horz. correlation length: 30.516079042533505 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=10: 1042, horz. correlation length: 30.462295122138265 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=15: 1144, horz. correlation length: 30.633533279802705 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=20: 1162, horz. correlation length: 30.652330183269058 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=25: 1241, horz. correlation length: 30.590440722056904 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=30: 1325, horz. correlation length: 30.647440906652236 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src

11[1.8838913944769675, 1.8834840454047588, 1.882679828900221, 1.8814994047647844, 1.8799730161881838, 1.8781396272253383, 1.8760458258719097, 1.8737445337549588, 1.871293568620661, 1.8687541083324135, 1.8661891050883037, 1.8636616963246355, 1.8612336547393573, 1.8589639145818362, 1.8569072053753002, 1.8551128180961154, 1.8536235229834235, 1.852474652938392, 1.8516933621138485, 1.8512980658832343]
1.867533167583232

┌ Info: Data points at z=100: 895, horz. correlation length: 0.5367513320764468 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Smoothed horz. correlation length at z=5: 1.8838913944769675
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=10: 1.8834840454047588
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=15: 1.882679828900221
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=20: 1.8814994047647844
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=25: 1.8799730161881838
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=30: 1.8781396272253383
└ @ DIVAnd C:\Users\HourstonH\.julia\p

┌ Info: Data points at z=5: 283, horz. correlation length: 3.1838755548717517 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=10: 286, horz. correlation length: 6.544648569665416 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=15: 299, horz. correlation length: 6.563272586098665 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=20: 302, horz. correlation length: 2.7008464890260733 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=25: 317, horz. correlation length: 6.697275697920154 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=30: 355, horz. correlation length: 6.44836331726269 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:107

12[5.574081000741981, 5.574255487015562, 5.574600000210942, 5.575105747742589, 5.575759851243623, 5.576545706085669, 5.577443439787086, 5.578430453005507, 5.5794820246149515, 5.580571961160877, 5.581673270745897, 5.582758842028565, 5.583802110365673, 5.584777695006823, 5.585661993456799, 5.586433721461049, 5.587074389371236, 5.587568707776434, 5.587904917150301, 5.588075037821216]
5.581100317839638

...

303[0.7441159785220278, 0.7441159778756372, 0.744115976635223, 0.7441159749012758, 0.7441159728142698, 0.7441159705432819, 0.7441159682722942, 0.7441159661852882, 0.744115964451341, 0.7441159632109268, 0.7441159625645362]
0.744115970543282

┌ Info: Data points at z=95: 55925, horz. correlation length: 0.6830182939572105 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=100: 54927, horz. correlation length: 0.33060919085119356 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Smoothed horz. correlation length at z=50: 0.7441159785220278
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=55: 0.7441159778756372
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=60: 0.744115976635223
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=65: 0.7441159749012758
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=70: 0.7441159728142698
└ @ DIVAn

┌ Info: Data points at z=50: 32201, horz. correlation length: 1.1929733008810204 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=55: 32124, horz. correlation length: 1.699062561297665 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=60: 31800, horz. correlation length: 1.8953104397783445 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=65: 31618, horz. correlation length: 1.9833671712614804 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=70: 31328, horz. correlation length: 1.9687813500863325 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=75: 30941, horz. correlation length: 1.3745637400241268 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDB

304[1.6103915485373475, 1.6103915477114608, 1.6103915461265959, 1.6103915439111485, 1.6103915412446015, 1.6103915383429828, 1.610391535441364, 1.610391532774817, 1.6103915305593695, 1.6103915289745045, 1.6103915281486179]
1.6103915383429823


┌ Info: Data points at z=95: 28987, horz. correlation length: 1.348815154643383 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Data points at z=100: 28403, horz. correlation length: 1.16324290337627 (preliminary)
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1072
┌ Info: Smoothed horz. correlation length at z=50: 1.6103915485373475
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=55: 1.6103915477114608
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=60: 1.6103915461265959
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=65: 1.6103915439111485
└ @ DIVAnd C:\Users\HourstonH\.julia\packages\DIVAnd\eDBbE\src\fit.jl:1100
┌ Info: Smoothed horz. correlation length at z=70: 1.6103915412446015
└ @ DIVAnd C

In [55]:
println(size(lenxy_mtx))

println(lenxy_mtx[:,1])

(30, 4)
Float32[1.8675332, 0.45356184, 1.0280122, 0.47871667, 1.0742917, 2.0678537, 1.3863764, 0.7357964, 1.6592437, 1.3317031, 1.3576366, 1.6301674, 0.8514943, 0.8879059, 0.32201546, 2.1122344, 3.703967, 0.1775403, 0.57338935, 0.52155894, 4.165799, 0.3576409, 8.152119, 1.0252799, 1.154806, 0.9546407, 3.5277472, 6.3400025, 0.87221533, 1.0210018]


In [58]:
# Convert 2d array to dataframe
dict = Dict("Year" => years, szns[1] => lenxy_mtx[:,1], 
    szns[2] => lenxy_mtx[:,2], szns[3] => lenxy_mtx[:,3], 
    szns[4] => lenxy_mtx[:,4])

# dict = Dict("Year" => years)

lenxy_df = DataFrame(dict)

,AMJ,JAS,JFM,OND,Year
,Float32,Float32,Float32,Float32,Int64
1,5.5811,0.980336,1.86753,0.319341,1991
2,0.447714,1.06999,0.453562,2.78672,1992
3,8.24734,1.03703,1.02801,1.177,1993
4,0.800049,0.80146,0.478717,1.17032,1994
5,0.458846,4.46269,1.07429,1.1379,1995
6,1.27802,0.80819,2.06785,1.12516,1996
7,1.28001,1.33137,1.38638,27.8413,1997
8,0.467684,1.18978,0.735796,1.4791,1998
9,1.27001,0.927296,1.65924,1.14335,1999


In [60]:
# Write dataframe to csv file
df_filename = "Oxy_fithorzlen_mean_lenxy_100m.csv"
CSV.write(df_filename, lenxy_df)

"Oxy_fithorzlen_mean_lenxy_100m.csv"

In [34]:
# df[!, :Depth_m]

195162-element Vector{Float64}:
   1.3
   5.1
   1.0
   4.9
   1.3
   5.0
 387.8
 317.3
   1.3
   4.9
 345.2
 288.4
   1.3
   ⋮
   2.97781
   9.032557
  25.012255
  58.952576
  88.02561
 100.0307
 126.02275
 142.98524
 196.046
 271.00195
 381.99762
 516.0047599999998

In [35]:
# # Find location of min and max of our data
# println(findmin(df[!, :Longitude]), findmax(df[!, :Longitude]))
# println(findmin(df[!, :Latitude]), findmax(df[!, :Latitude]))
# println(findmin(df[!, :Depth_m]), findmax(df[!, :Depth_m]))
# println(findmin(df[!, :Value]), findmax(df[!, :Value]))

(-145.0, 75553)(-117.28833, 193639)
(30.18, 193892)(53.988167, 186606)
(0.9, 104393)(4240.074000000001, 79776)
(2.6141999, 194605)(435.8, 181537)


Maybe I need the depth range to be within the range of all profiles in the observational data set, not just some profiles.

In [39]:
# xx = deleteat!(lenxy_oxy, findall(x->x == NaN64, lenxy_oxy))

println(lenxy_oxy)
println(mean(lenxy_oxy))

# lenx = 500e3;

[2.581373954679853, 2.5809811543905266, 2.580205254214584, 2.579065413320741, 2.5775897725934622, 2.5758147544352488, 2.573784157377197, 2.5715480686305963, 2.569161622237169, 2.5666836342327386, 2.5641751491407403, 2.56169793409929, 2.5593129579688743, 2.55707889286581, 2.5550506747435735, 2.5532781579466954, 2.5518048961555935, 2.5506670789058714, 2.5498926489935245, 2.5495006216675398]
2.5654333399299816


In [41]:
[1991:1:2020;]

30-element Vector{Int64}:
 1991
 1992
 1993
 1994
 1995
 1996
 1997
 1998
 1999
 2000
 2001
 2002
 2003
    ⋮
 2009
 2010
 2011
 2012
 2013
 2014
 2015
 2016
 2017
 2018
 2019
 2020

In [45]:
xx = [1,2,3,4,5]

xx[1] = 55

print(xx)

[55, 2, 3, 4, 5]